In [ ]:
from flask import Flask, request, jsonify
import joblib
import numpy as np  # Necessário para manipular os arrays de índices

app = Flask(__name__)

# --- Carregamento do Modelo ---
try:
    data = joblib.load('modelo_b2w_rating_sentimento.pkl')
    model = data['model']
    vectorizer = data['vectorizer']
    print("Modelo carregado com sucesso!")
except Exception as e:
    print(f"Erro crítico: {e}")
    model = None
    vectorizer = None

LABEL_MAP = {
    "negative": "Negativo",
    "neutral": "Neutro",
    "positive": "Positivo"
}

@app.route('/predict', methods=['POST'])
def predict():
    try:
        dados = request.get_json()
        texto = dados.get('text') if dados else None

        if not texto or len(texto.strip()) < 5:
            return jsonify({"erro": "Texto não fornecido ou muito curto."}), 400

        # 1. Transformar o texto
        X = vectorizer.transform([texto])

        # 2. Fazer a previsão
        prediction_label = model.predict(X)[0]
        proba = float(model.predict_proba(X).max())

        # --- BLOCO NOVO: EXPLICABILIDADE ---
        palavras_relevantes = []

        # Só podemos explicar se o modelo tiver coeficientes lineares
        if hasattr(model, 'coef_'):
            # Pega os nomes de todas as palavras do vocabulário
            feature_names = vectorizer.get_feature_names_out()

            # Pega os índices das palavras que EXISTEM na frase enviada
            # X é uma matriz esparsa; nonzero() retorna (indices_linhas, indices_colunas)
            indices_palavras_na_frase = X.nonzero()[1]

            # Identifica qual linha de coeficientes usar
            # Se for multiclasse, model.coef_ tem formato (n_classes, n_features)
            # Precisamos achar o índice numérico da classe prevista (ex: 0, 1 ou 2)
            classe_idx = np.where(model.classes_ == prediction_label)[0][0]

            # Pega os pesos correspondentes à classe prevista
            # Se for binário, a lógica é um pouco diferente, mas assumindo multiclasse aqui:
            if model.coef_.ndim > 1:
                pesos_classe = model.coef_[classe_idx]
            else:
                # Fallback para binário simples (se necessário)
                pesos_classe = model.coef_[0] if classe_idx == 1 else -model.coef_[0]

            # Monta a lista de palavras e seus pesos
            for idx in indices_palavras_na_frase:
                peso = pesos_classe[idx]
                palavra = feature_names[idx]

                # Opcional: Filtrar apenas palavras com algum impacto relevante
                palavras_relevantes.append({
                    "palavra": palavra,
                    "peso": round(float(peso), 4)
                })

            # Ordena: As palavras com maior peso positivo aparecem primeiro (ajudaram na decisão)
            palavras_relevantes = sorted(palavras_relevantes, key=lambda x: x['peso'], reverse=True)

        # -----------------------------------

        return jsonify({
            "previsao": LABEL_MAP.get(prediction_label, "Desconhecido"),
            "probabilidade": round(proba, 2),
            "analise_pesos": palavras_relevantes  # <--- Nova chave no retorno JSON
        })

    except Exception as e:
        import traceback
        traceback.print_exc()
        return jsonify({"erro": str(e)}), 500

####if __name__ == '__main__':
    app.run(host="0.0.0.0", port=5000)


#################################################################
#################################################################
#################################################################

# ... (Mantenha todo o seu código anterior de importação e carregamento do modelo) ...

def testar_localmente(texto):
    """
    Função para testar o modelo direto no terminal sem usar JSON/Flask
    """
    if not texto: return

    print(f"\n{'='*40}")
    print(f"ANALISANDO: \"{texto}\"")
    print(f"{'='*40}")

    # 1. Previsão
    X = vectorizer.transform([texto])
    pred_label = model.predict(X)[0]
    proba = model.predict_proba(X).max()

    nome_classe = LABEL_MAP.get(pred_label, pred_label)
    print(f"RESULTADO: {nome_classe.upper()} (Confiança: {proba:.1%})\n")

    # 2. Explicabilidade (Pesos)
    if hasattr(model, 'coef_'):
        feature_names = vectorizer.get_feature_names_out()
        indices_frase = X.nonzero()[1]

        # Lógica para pegar os pesos da classe prevista
        idx_classe = np.where(model.classes_ == pred_label)[0][0]

        # Se for multiclasse ou binário, ajusta acesso ao array
        if model.coef_.ndim > 1:
            pesos = model.coef_[idx_classe]
        else:
            # Se for binário simples (0 e 1), classe 1 é positivo, classe 0 inverte o sinal
            pesos = model.coef_[0] if idx_classe == 1 else -model.coef_[0]

        # Montar lista de (palavra, peso)
        lista_pesos = []
        for idx in indices_frase:
            lista_pesos.append((feature_names[idx], pesos[idx]))

        # Ordenar: Palavras que mais ajudaram primeiro
        lista_pesos.sort(key=lambda x: x[1], reverse=True)

        print("--- POR QUE ESSE RESULTADO? ---")
        print(f"{'PALAVRA':<20} | {'PESO (IMPACTO)':<15}")
        print("-" * 35)

        for palavra, peso in lista_pesos:
            sinal = "+" if peso > 0 else ""
            print(f"{palavra:<20} | {sinal}{peso:.4f}")

    print("\n")


# --- BLOCO PRINCIPAL (SUBSTITUA AS ULTIMAS LINHAS POR ISSO) ---
if __name__ == '__main__':
    # Se quiser rodar o servidor, descomente a linha abaixo:
    # app.run(host="0.0.0.0", port=5000)

    # MODO DE TESTE MANUAL
    while True:
        frase_usuario = input("Digite uma frase para testar (ou 'sair'): ")
        if frase_usuario.lower() == 'sair':
            break
        testar_localmente(frase_usuario)

Modelo carregado com sucesso!
Digite uma frase para testar (ou 'sair'): este produto deu pau demais

ANALISANDO: "este produto deu pau demais"
RESULTADO: NEGATIVO (Confiança: 48.2%)

--- POR QUE ESSE RESULTADO? ---
PALAVRA              | PESO (IMPACTO) 
-----------------------------------
produto              | +1.5445
deu                  | +0.5170
demais               | +0.3715
pau                  | +0.2501
produto deu          | -0.1914


Digite uma frase para testar (ou 'sair'): deu problema demais

ANALISANDO: "deu problema demais"
RESULTADO: NEGATIVO (Confiança: 40.7%)

--- POR QUE ESSE RESULTADO? ---
PALAVRA              | PESO (IMPACTO) 
-----------------------------------
deu                  | +0.5170
demais               | +0.3715
deu problema         | +0.0158
problema             | -0.2293


Digite uma frase para testar (ou 'sair'): desde ontem chegou e nao testei

ANALISANDO: "desde ontem chegou e nao testei"
RESULTADO: NEGATIVO (Confiança: 42.2%)

--- POR QUE ESSE RESUL